# 异步IO：asyncio（以下内容仅3.7+）

异步，是指在发起一个IO操作时，无需等其结束，可以同时做其他事情，在事情完成后，可得到通知。

asyncio模块包通过**async**/**await**语法实现协程代码。

asyncio是作为Python高并发网络，网络服务器，数据库连接，序列任务分发等异步框架。

异步程序要比经典的序列化程序复杂。

## **理解异步、协程**

现在我们讲了一些异步的使用，是时候解释一些概念了

**同步、异步、阻塞、非阻塞**四个词语之间的联系

- 首先要明确，前两者后后两者并不是一一对应的，它们不是在说同一件事情，但是非常类似，容易搞混
- 一般我们说异步程序是非阻塞的，而同步既有阻塞也有非阻塞的
- 非阻塞是指一个任务没做完，没有必要停在那里等它结束就可以开始下一个任务，保证一直在干活没有等待；阻塞就相反是一件事完全结束才开始另一件事
- 在非阻塞的情况下，同步与异步都有可能，它们都可以在一个任务没结束就开启下一个任务。而二者的区别在于：（且称正在进行的程序为主程序）当第一个程序做完的时候（比如网络请求终于相应了），会自动通知主程序回来继续操作第一个任务的结果，这种是异步；而同步则是需要主程序不断去问第一个程序是否已经完成。
- 四个词的区别参考[知乎回答](https://www.zhihu.com/question/19732473)

**协程与多线程的区别**

- 在非阻塞的情况下，多线程是同步的代表，协程是异步的代表。二者都开启了多个线程
- 多线程中，多个线程会竞争谁先运行，一个等待结束也不会去通知主程序，这样没有章法的随机运行会造成一些资源浪费
- 而协程中，多个线程（称为微线程）的调用和等待都是通过明确代码组织的。协程就像目标明确地执行一个又一个任务，而多线程则有一些彷徨迷茫的时间

**两种异步**

- 前面几节涉及到两种异步，一种是`await`只使用一个线程就可以实现任务切换，另一种是开启了多个线程，通过线程调度实现异步
- 一般只用一个线程将任务在多个函数之间来回切换，是使用yield生成器实现的，例子可以看[这篇文章最后生产消费者例子](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432090171191d05dae6e129940518d1d6cf6eeaaa969000)

**多进程、多线程、异步擅长方向**

- 异步和多线程都是在IO密集型任务上优势明显，因为它们的本质都是在尽量避免IO等待时间造成的资源浪费。而多进程可以利用多核优势，适合CPU密集型任务
- 相比于多线程，异步更适合每次等待时间较长、需要等待的任务较多的程序。因为多线程毕竟要创建新的线程，线程过多使线程竞争现象更加明显，资源浪费也就更多。如果每个任务等待时间过长，等待时间内势必开启了非常多任务，非常多线程，这时使用多线程就不是一个明智的决定。而异步则可以只开启一个线程在各个任务之间有条不紊进行，即能充分利用CPU资源，又不会影响程序运行效率

# 协程与任务(coroutine and task)

本处概要：最外层asyncio API接口的协程和任务使用说明。

## 协程

协程corotine可通过在对象前声明`async/await`来定义，是异步编程的首选方式。

举个如下片段的例子(Python 3.7+) ，打印 "Hello" ,等待1s，然后打印 "world"

In [1]:
import asyncio

async def main():
    print("hello")
    await asyncio.sleep(1)
    print('world')
    
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

如果不通过run()函数，而是直接调用main()，则不会被执行

如果要运行一个协程，asyncio提供三种机制。


- [`asyncio.run()`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.run) 函数运行最顶层的入口函数，如上例举所指向“main()” 函数 

- 协程等待期间，下面的代码片段会在1s之后打印“hello”，再继续等待2s之后，会打印"world"。


In [1]:
import asyncio
import time

async def say_after(delay,what):
    print(f"{time.strftime('%X')}")
    await asyncio.sleep(delay)
    print(f"{time.strftime('%X')}:{what}")

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1,'hello')

    await say_after(2,'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

- 通过[`asyncio.create_task()`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.create_task) 函数封装协程作为asyncio [`Tasks`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.Task)的并发任务.

  修改以上用例，运行两次 `say_after` 实现协程并发:

In [ ]:
async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

# 可等待对象（awaitable）

我们描述一个可等待对象是看它是否用`await`表达式修饰。很多asyncio APIs是设计为可等待对象。

有三种类型的可等待对象： `coroutines`, `Tasks` 和 `Futures`。

## Coroutines

Python的协程是可等待对象(awaitable)，因此也能被其他协程`coroutine`等待。

In [ ]:
import asyncio

async def nested():
    return 42

async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    nested()

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

asyncio.run(main())

**要点**: 关于"协程"`coroutine`的术语在本文表示两种相关性紧密的概念。

- 协程函数:async def修饰的函数；

- 协程对象:通过调用协程函数返回的对象。

asyncio也支持传统的基于生成器的协程。

## Task

Task项是用来调度协程并发。

当一个协程被任务函数[`asyncio.create_task()`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.create_task) 封装，那么协程将会自动进入调度运行。

In [2]:
import asyncio


async def nested():
    return 42


async def main():
    # Schedule nested() to run soon concurrently
    # with "main()".
    task = asyncio.create_task(nested())

    # "task" can now be used to cancel "nested()", or
    # can simply be awaited to wait until it is complete:
    await task


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

## Futures

未来对象是一个特殊的最底层可等待对象，也是作为异步操作的**最终结果**

当一个`future`对象被`await`时，表示协程会持续等待，直到`future`对象的异步操作执行完毕。

`Future`对象在syncio中需要允许基于回调代码用于`async/await`表达式。

通常而言，在应用层编程中不需要创建`Future`对象。

`Future`对象，有时候会被某些库和asyncio APIs体现是可`await`的。

In [ ]:
async def main():
    await function_that_returns_a_future_obejct()
    # this is also valid:
    await asyncio.gather(function_that_returns_a_future_obejct(),
                         some_python_coroutine())

[`loop.run_in_executor()`](https://docs.python.org/3.7/library/asyncio-eventloop.html#asyncio.loop.run_in_executor)方法是一个在底层函数返回`Future`对象的例子。

# 运行一个asyncio程序

`asyncio.run(coro,*,debug=False)`



参考 https://www.imooc.com/article/263959
https://docs.python.org/3/library/asyncio.html